In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install --upgrade pymupdf


In [4]:
import os
import fitz  # PyMuPDF
import pandas as pd
import logging


# Folder with all copied SAM PDFs
pdf_folder = "/content/drive/My Drive/AI4ALL PROJECT/SAM_All"

# The expected emotions in order
emotion_fields = ["satisfaction", "boredom", "horrible", "calm", "funny"]

# Store results here
data = []

# Loop through all PDF files
for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_folder, filename)

        try:
            with fitz.open(file_path) as doc:
                logging.getLogger("fitz").setLevel(logging.ERROR)
                text = ""
                for page in doc:
                    text += page.get_text()

            # Extract numbers from text (we assume scores are integers 1–10)
            lines = text.splitlines()
            scores = []
            for line in lines:
                try:
                    num = int(line.strip())
                    if 1 <= num <= 10:
                        scores.append(num)
                except:
                    continue


            if len(scores) >= 5:
                # Map scores to emotion labels
                emotion_scores = scores[:5]
                # Swap horrible and calm scores if their positions are flipped
                emotion_scores[2], emotion_scores[3] = emotion_scores[3], emotion_scores[2]

                emotion_dict = dict(zip(emotion_fields, emotion_scores))

                # Choose dominant emotion
                max_score = max(emotion_scores)
                max_index = emotion_scores.index(max_score)
                dominant_emotion = emotion_fields[max_index]

                record = {
                    "pdf_file": filename,
                    **emotion_dict,
                    "dominant_emotion": dominant_emotion.capitalize()
                }
                data.append(record)
            else:
                print(f"Not enough scores in: {filename}")

        except Exception as e:
            print(f"Error reading {filename}: {e}")

# Create DataFrame
df_sam = pd.DataFrame(data)

# Save to a CSV for training later
# df_sam.to_csv("/content/drive/My Drive/AI4ALL PROJECT/sam_labels.csv", index=False)


ModuleNotFoundError: No module named 'fitz'